# Тетрадка содержит запуск функций поиска аномалий в базах на тестовых данных

In [1]:
%load_ext autoreload
import os
import sys
import pandas as pd
from IPython.display import display
sys.path.append(os.path.join('../src/utils'))
sys.path.append(os.path.join('../src/'))
import streamlit_funcs

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

DB1_FOLDER = '../test_data/db1'
DB2_FOLDER = '../test_data/db2'

В тестовом датасете в названиях колонок точка вместо подчеркивания. Исправляем это

In [3]:
def get_filenames_from_dir(path, include_subdirs=True, file_extensions=None):
    if include_subdirs:
        files_list = []
        for root, _, files in os.walk(path):
            files_list += [os.path.join(root, f) for f in files if os.path.isfile(os.path.join(root, f))]
    else:
        files_list = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

    files_list = [f for f in files_list if f.split('.')[-1] in file_extensions]

    return files_list

In [4]:
def fix_colnames(path):
    print(path)
    sep=','
    if 'ref' in path:
        sep = ';'
    df = pd.read_csv(path, sep=sep)
    df.columns = [col.replace('.', '_') for col in df.columns]
    df.to_csv(path, index=False, sep=sep)

In [5]:
### !!!! РАСКОММЕНТИТЬ КОД ЕСЛИ В СЛЕДУЮЩЕЙ ЯЧЕЙКЕ ПОЯВИТСЯ ПРИ ЗАПУСКЕ НА ТЕСТОВЫХ ДАННЫХ KeyError: 'id_fish'

# test_data_files = get_filenames_from_dir('../test_data/', include_subdirs=True, file_extensions='csv')
# for filepath in test_data_files:
#     fix_colnames(filepath)

In [6]:
db1 = streamlit_funcs.load_db1(DB1_FOLDER)

In [7]:
ext1, ext2 = streamlit_funcs.load_db2(DB2_FOLDER)

In [8]:
db2 = streamlit_funcs.get_db2(ext1, ext2)

# Расхождения между базами по данным с одинаковыми ключами

Объединяем базы по ключу ['id_ves', 'date', 'id_fish'], считаем суммарный улов и суммарный продукт по этому ключу

Идея - не весь улов сдается в порту

Сравниваем (считаем в процентах разность между уловом и продуктом, поделенными на улов, 'отклонение внесенного от выловленного, %')

В базе 2 вес продукта похоже внесен с ошибками, поэтому для расчета 'отклонение внесенного от выловленного, %' брали не только оригинал, но также его делили на 100 и 1000 и брали минимальное из отклонений

Также учитывали возможное смещение между выловом и сдачей на базу, настраивается параметром shift в streamlit_funcs.aggregate_db1_db2_table

Смотрим не только улов в данный день, но в предыдущие (согласно значению shift; shift=3 значит, что смотрим также улов в предыдущие записи в базе 3 дня)

# Таблицы аномалий

In [9]:
%autoreload 2
test_joined = streamlit_funcs.aggregate_db1_db2_table(db1, db2, threshold=25, shift=3)

In [10]:
test_joined

,id судна (id_ves),дата,id рыбы,назавание рыбы,улов,внесено в базу (размерность неизвестна),внесено в базу (коррекция 1/1000),внесено в базу (коррекция 1/100),"отклонение внесенного от выловленного, %","порог отклонения, %",является ли подозрительным
38115,1510,2022-01-11,292,треска,670.0,0,0.000,0.00,100.00,25,1
38110,1510,2022-01-08,292,треска,500.0,0,0.000,0.00,100.00,25,1
38112,1510,2022-01-08,403,навага,30.0,0,0.000,0.00,100.00,25,1
38113,1510,2022-01-08,451,камбалы дальневосточные,300.0,0,0.000,0.00,100.00,25,1
38114,1510,2022-01-08,677,бычки,200.0,0,0.000,0.00,100.00,25,1
...,...,...,...,...,...,...,...,...,...,...,...
18305,1383,2022-01-24,617,зубатка пестрая,228.0,163,0.163,1.63,28.51,25,1
24712,1424,2022-03-19,112,палтус белокорый,1316.0,947,0.947,9.47,28.04,25,1
51389,1603,2022-04-24,292,треска,5602.0,4117,4.117,41.17,26.51,25,1
30838,1460,2022-02-04,147,окунь золотистый,2786.0,3494,3.494,34.94,25.41,25,1


В начале таблицы видим судно 1510, у которого есть улов трески, наваги, бычков и т.д., но в базу записан 0

Также видим сильные систематические отклонения для судна 1289:

In [17]:
thresh = 50
test_joined[test_joined['отклонение внесенного от выловленного, %'] > thresh].sort_values(by='id судна (id_ves)')

,id судна (id_ves),дата,id рыбы,назавание рыбы,улов,внесено в базу (размерность неизвестна),внесено в базу (коррекция 1/1000),внесено в базу (коррекция 1/100),"отклонение внесенного от выловленного, %","порог отклонения, %",является ли подозрительным
825,1286,2022-04-25,451,камбалы дальневосточные,359.00,1617,1.617,16.17,95.50,25,1
1239,1289,2022-04-22,292,треска,17105.70,34210,34.210,342.10,98.00,25,1
1240,1289,2022-04-22,610,зубатка синяя,570.90,1140,1.140,11.40,98.00,25,1
1243,1289,2022-04-22,729,палтус синекорый,1028.16,2056,2.056,20.56,98.00,25,1
1122,1289,2022-03-27,147,окунь золотистый,292.50,929,0.929,9.29,96.82,25,1
...,...,...,...,...,...,...,...,...,...,...,...
59466,1654,2022-01-02,294,краб-стригун опилио,341.00,655,0.655,6.55,92.08,25,1
59860,1655,2022-05-12,88,окунь-клювач,25706.00,10000,10.000,100.00,61.10,25,1
59974,1656,2022-04-09,400,минтай,309309.00,618690,618.690,6186.90,98.00,25,1
60221,1659,2022-04-14,204,сельдь тихоокеанская,54388.00,106868,106.868,1068.68,96.49,25,1


In [15]:
anomaly_ves = set(test_joined['id судна (id_ves)'])
ves_from_db1 = set(db1['id_ves'])

print('Доля судов, для которых наблюдались различия в улове и весе продукта из базы 2 от всех из базы 1:\n', round(len(anomaly_ves) / len(ves_from_db1), 2))

Доля судов, для которых наблюдались различия в улове и весе продукта из базы 2 от всех из базы 1:
 0.16


In [20]:
# Примеры для id_ves [1289, 1510]
example_ids = [1289, 1510]
db1[db1['id_ves'].isin(example_ids)]

,id_ves,id_region,id_fish,catch_volume,id_regime,permit,id_own,fish,catch_date
128,1289,108,714,26.970,97,2774,1047,камбала-ерш,2022-01-01
129,1289,108,617,43.484,97,2774,1047,зубатка пестрая,2022-01-01
130,1289,108,610,71.197,97,2774,1047,зубатка синяя,2022-01-01
131,1289,108,292,9884.370,15,2774,1047,треска,2022-01-01
538,1289,108,292,14998.980,15,2774,1047,треска,2022-01-02
...,...,...,...,...,...,...,...,...,...
115530,1289,437,88,436.450,97,2774,1047,окунь-клювач,2022-05-26
115531,1289,437,292,43737.000,15,2774,1047,треска,2022-05-26
115532,1289,437,408,4477.200,15,2774,1047,пикша,2022-05-26
115533,1289,437,610,920.700,97,2774,1047,зубатка синяя,2022-05-26


Также идеи: использовать модели машинного обучения для прогноза улова по характеристикам: региону, режиму, произведенным продуктам и проч. 

Сравнивать прогноз с реальными значениями. При существенных отклонениях маркировать как подозрительные. 

Модели обучены, но не внедрены, код можно посмотреть в тетрадке db1_anomaly_train_model.ipynb. Качество моделей проверялось на кросс-валидации, метрики roc-auc (для классификации) и r2 (для регрессии)

В дальнейшем суда, маркированные как подозрительные, можно прицельно искать по базе 2 по весу вылова для конкретных рыб. При их отсутствии в базе подозрение усиливается